In [ ]:
# Importing numpy and functions
import numpy as np
import helpers as h
from implementations import *

# Loading the data
X, xHeader, Y, yHeader, indexedX, indexedXheader, indexedY, indexedYheader = loadTrainingData()
print('')
# Cleaning/feature engineering the data
yClean, xClean, xHeaderClean, removedFeatures = dataCleaning(Y,X,xHeader)
print('')
# Making a balanced data set to force the model to not just predict negatively all the time
yBalanced, xBalanced, balancePrior = balanceData(yClean,xClean)
print('')
# Adding dummy variables and replacing the remaining invalid values by the mean
tx = makeTrainingData(xBalanced)
print(f'The resultant dataarray tx has shape {tx.shape}')

In [ ]:
# Initializing the parameters at zero and setting some constants
initial_w = np.zeros(tx.shape[1])
K = 5
gamma = 0.01
max_iter = 1000
lambda_ = 0.1
# Training a model with gradient descent with momentum
#w_gd_m, loss_gd_m = k_fold_cross_validation(yBalanced,tx,K,initial_w,max_iter,gamma, mse_gd_momentum, compute_loss)
print('')
# Training a model with logistic reg with momentum
#w_logistic, loss_logistic = k_fold_cross_validation(yBalanced,tx,K,initial_w,max_iter,gamma)

# Training a model with logistic reg with momentum

setLambdaReg = lambda (yBalanced,tx,K,initial_w,max_iter,gamma): penalized_logistic_regression(y, tx, w, lambda_=0.1)
w_logistic, loss_logistic = max_cross_valid(yBalanced,tx,K,initial_w,max_iter,gamma)


In [ ]:
# Loading the test data
xTest, xIndexedHeader = loadData('./Data/x_test.csv')
print(xTest.shape)

In [ ]:
"""
def makePredictions(w,xTest,xHeader,xHeaderFeaturesRemoved, prior=1.0):
    ''' Function making predictions based on provided parameters and data
    Args:
        w: (d,) array with the parameters
        x: (N,D) array with the data
        xHeader: (D,) array with all the features
        xHeader: (d,) array with the features that are actually used
        prior: float denoting the probability of a random sample being in the model training data
    Returns:
        (N,) boolean array of the predictions
    '''
    standardX = standardizeData(xTest)
    removedFeaturesX = standardX[:,np.nonzero(np.isin(xHeader, xHeaderFeaturesRemoved))[0]]
    predictionSet = makeTrainingData(removedFeaturesX)
    probabilities = prior * logistic(predictionSet@w) # The prob of the model being applicable times the prob from the model
    return (np.sign(probabilities-0.5)+1)/2 # Shifting the probs to be negative for negative preds, and vice versa, taking the sign, shifting the preds up to be zero or two, diving by to so the preds are zero or one
"""
# Making predictions
pred_gd = makePredictions(w_logistic,xTest[:,1:],xHeader,xHeaderClean,balancePrior)
pred_logistic = makePredictions(w_logistic,xTest[:,1:],xHeader,xHeaderClean)
# Counting predicted positive cases
print(np.sum(pred_gd),np.sum(pred_logistic))


In [ ]:
pred_logistic[pred_logistic == 0] = -1
h.create_csv_submission(xTest[:,0],pred_logistic,'./Predictions/balancedDataNoPrior.csv')